In [1]:
import matplotlib.pyplot as plt

ImportError: No module named matplotlib.pyplot

In [2]:
from pandora.astro import Mags

In [3]:
from fase.fileio import Table

In [4]:
from fase.fileio import Image

In [5]:
from fase.fileio import Spectrum

In [10]:
from astropy.cosmology import FlatLambdaCDM

In [7]:
import numpy as np

In [8]:
from GMfunctions import *

In [9]:
import os

# Function to read magnitudes from the evolved model

In [11]:
def getUV_NewModel(templ_start,delta_t):
    jump_n_model = int((delta_t/0.1)+0.5)
    templ_end = templ_start + jump_n_model

    #get tau of templ_start and compare it to the tau of templ_end

    if templ_end<=1 or ima.getRow(templ_end)[tao_position] != ima.getRow(templ_start)[tao_position]:
        ERROR= True
    else:
        ERROR=False

    if not ERROR:
        #print "numero di modelli da saltare:", jump_n_model, "--> delta t= ",delta_t
        flx_all = ima.getRow(templ_end)
        flx = flx_all[0:last_lamb+1]
        spec = Spectrum((lambdas,flx))
        u = spec.getMag("U_Bessel",1,mag)
        v = spec.getMag("V_Buser",1,mag)
        logmass = flx_all[mass_position]
    else:
        u = -99.
        v = -99.
        logmass = -99.
    return u,v,logmass,ERROR

# Read variables

In [16]:
match = Table("../data/Data_Sample_V7.fits",hdu=8)
# image containing PEGASE models for SFH with no quenching
ima = Image("../data/models_san_new_wind_abs.fits")
mag = Mags()

cosmo = FlatLambdaCDM(H0=70,Om0=0.3)

lambdas_all = ima.getRow(1)
last_lamb = lambdas_all.argmin()-1
lambdas = lambdas_all[0:last_lamb+1]

# position in pixels where the 
# parameters are stored in ima
tao_position  = last_lamb+1
age_position  = last_lamb+2
sfr_position  = last_lamb+6
mass_position = last_lamb+7
metallicity_position = last_lamb+5

In [17]:
z       = match.getCol("zspec_1")
templ   = match.getCol("TEMPL")
mag_u   = match.getCol("M_Uj")
mag_v   = match.getCol("M_V")
uv      = mag_u - mag_v
lgmass  = match.getCol("log_stellar_mass")

mtau = match.getCol("TAU_2")
mage = match.getCol("AGE_2")
mindex = match.getCol("num_1")

In [18]:
# zlow 	 zhigh 	 z_median 	 t_median 
# 4.000000000000000222e-01 5.000000000000000000e-01 4.659999907016754150e-01 8.668469387307764507e+00
# 5.000000000000000000e-01 5.999999999999999778e-01 5.529999732971191406e-01 8.056404053468638082e+00
# 5.999999999999999778e-01 6.999999999999999556e-01 6.470000147819519043e-01 7.465174918806797955e+00
# 6.999999999999999556e-01 8.000000000000000444e-01 7.491000294685363770e-01 6.894509193135578862e+00
# 8.000000000000000444e-01 9.000000000000000222e-01 8.486499786376953125e-01 6.400014398583118158e+00
# 9.000000000000000222e-01 1.000000000000000000e+00 9.408000111579895020e-01 5.989489134132708337e+00
# 1.000000000000000000e+00 1.300000000000000044e+00 1.090000033378601074e+00 5.406702162694932312e+00

# Computing median redshifts and cosmic time for evolution

In [20]:
zint_low  = [0.4,0.5,0.6,0.7,0.8,0.9,1.0]
zint_high = [0.5,0.6,0.7,0.8,0.9,1.0,1.1]
zetas = np.zeros(len(zint_low))
times = np.zeros(len(zint_low))
for i in range(len(zetas)):
    z_bin = np.where((z>zint_low[i])&(z<zint_high[i]))
    zetas[i] = np.median(z[z_bin])
    times[i] = cosmo.age(z=zetas[i]).value

In [21]:
times

array([ 8.66094395,  8.06143064,  7.4719106 ,  6.9014859 ,  6.40691613,
        6.00713169,  5.58117456])

In [22]:
# median redshift and median cosmic time for redshift bins
# they have been computed in Bologna galaxies so that 
# they are slightly different from the matched catalogue with Milan

# After checked that everything is working change them in the actual ones
#zint_low, zint_high, zetas, times = np.loadtxt("delta_t.txt", unpack=True)

# Create output folder 

In [23]:
# this is where to save output files
#nome= "/Users/giorgiomanzoni/Desktop/test/"
nome="../VIPERS/SYNTHETIC"
if not os.path.isdir(nome):
    os.mkdir(nome)
nome += "/NO_QUENCH/"
if not os.path.isdir(nome):
    os.mkdir(nome)
print "Created folder: "+nome

Created folder: ../VIPERS/SYNTHETIC/NO_QUENCH/


# Actual evolution of galaxies - NO QUENCHING

In [24]:
# LOOP Z BIN START
#for t_start_pos in reversed(range(len(zetas))):   #6 5 4 3 2 1 0
for t_start_pos in [6]:
    
    print "z start = "+str(zint_low[t_start_pos])+" < z < "+str(zint_high[t_start_pos])
    
    z_bin = np.where((z>zint_low[t_start_pos])&(z<zint_high[t_start_pos]))
    z1013 = z[z_bin]
    templ1013 = templ[z_bin]
    uv1013 = uv[z_bin]
    v1013 = mag_v[z_bin]
    mass1013 = lgmass[z_bin]
    tau1013 = mtau[z_bin]
    age1013 = mage[z_bin]
    index1013 = mindex[z_bin]

    for t_end_pos in reversed(range(t_start_pos)):# if BACK: (6)--> 5,4,3,2,1,0
        if t_end_pos == t_start_pos:
            continue

        DELTAT = times[t_end_pos]-times[t_start_pos]


        synthetic_v = []
        synthetic_uv = []
        synthetic_mass = []
        synthetic_tau = []
        synthetic_age = []
        # I know that does not change from the real but if
        # I want to use np.where I have to manain the array to be of the same dimension (the same for taus)
        synthetic_templ = []    
        synthetic_index = []
        
        
        
        for i in range(len(z1013)):
            ERROR = False
            fluxes_all = ima.getRow(templ1013[i]+1)
            fluxes = fluxes_all[0:last_lamb+1]
            spectra = Spectrum((lambdas,fluxes))

            U_i  = spectra.getMag("U_Bessel",1,mag)
            V_i  = spectra.getMag("V_Buser",1,mag)
            MASS_i = fluxes_all[mass_position]
            UV_initial = U_i - V_i

            tao = fluxes_all[tao_position]
            age = fluxes_all[age_position]

            new_age = age + DELTAT
            #_______________________________________________________________
            U_f ,V_f, MASS_f,ERROR = getUV_NewModel(templ1013[i]+1,DELTAT)
            #_______________________________________________________________
            
            if ERROR:
                raise Exception('Something went wrong while computing \
                            the evolution of the model'+str(templ1013[i]+"for "+str(DELTAT)+" Gyr"))
                
            UV_final  = U_f  - V_f
            delta_UV = UV_final - UV_initial
            delta_V  = V_f - V_i
            #delta_MASS = MASS_f - MASS_i
            delta_MASS_ratio = MASS_f / MASS_i

            if not ERROR:
                synthetic_v.append(v1013[i]+delta_V)
                synthetic_uv.append(uv1013[i]+delta_UV)
                synthetic_mass.append(mass1013[i]+np.log10(delta_MASS_ratio))
                synthetic_tau.append(tau1013[i])
                synthetic_age.append(age1013[i]+DELTAT)
                synthetic_templ.append(templ1013[i])
                synthetic_index.append(index1013[i])
            else:
                synthetic_v.append(-99.)
                synthetic_uv.append(-99.)
                synthetic_mass.append(-99.)
                synthetic_tau.append(-99.)
                synthetic_age.append(-99.)
                synthetic_templ.append(-99.)
                synthetic_index.append(-99.)

 
        synthetic_uv = np.array(synthetic_uv)
        synthetic_v = np.array(synthetic_v)
        synthetic_mass = np.array(synthetic_mass)
        synthetic_tau = np.array(synthetic_tau)
        synthetic_age = np.array(synthetic_age)
        synthetic_templ = np.array(synthetic_templ)
        synthetic_index = np.array(synthetic_index)
        
        #SAVE SYNTHETIC GALAXIES IN A TXT FILE
        SYN_POINTS_filename = nome+"SYNTHETIC_points_%i%i_%i%i.txt"%(zint_low[t_start_pos]*10,zint_high[t_start_pos]*10,zint_low[t_end_pos]*10,zint_high[t_end_pos]*10)
        data_syn = [synthetic_uv,synthetic_v,synthetic_mass]
        data_syn = np.array(data_syn)
        np.savetxt(SYN_POINTS_filename,data_syn.T,header="syn_color_uv \t syn_mag_v \t syn_log_mass")


z start = 1.0 < z < 1.1


# Plotting

In [69]:
def read_synthetic_1011(zbin_low=0.40,zbin_high=0.50,quench_type=-99,folder='./SYNTHETIC/'):
    '''evolution starts always from bin 1.0 < z < 1.3 into the bin specified by the input parameters
    quench type can be [-99,0,1,2,3] that means respectively no quench (Sandage exponentially delayed SFH), 
    quench at the epoch of observation, 
    and quench randomly spread in interval times of 1,2,3 Gyrs after the epoch of observation. 
    The function returns the value of v,u-v points'''
    import pandas as pd
    
    a=str(int(zbin_low*10))
    b=str(int(zbin_high*10))
    z_end_label =a+b
    quench_type_label = ""
    
    if quench_type==-99:
        quench_type_label='NO_QUENCH'
    elif quench_type==0:
        quench_type_label='QUENCH_0'
    elif quench_type==1:
        quench_type_label='QUENCH_1'
    elif quench_type==2:
        quench_type_label='QUENCH_2'
    elif quench_type==3:
        quench_type_label='QUENCH_3'
    
    if folder[-1]=="/":
        folder = folder[:-1]

    data = pd.read_csv(folder+quench_type_label+'/SYNTHETIC_points_1011_'+z_end_label+'.txt',sep = ' ',usecols=[0,1],comment='#',names=['uv','v'])
    return np.array(data['v']),np.array(data['uv'])

In [72]:
read_synthetic_1011()

ImportError: No module named pandas

In [75]:
import pandas

ImportError: this version of pandas is incompatible with numpy < 1.12.0
your numpy version is 1.9.3.
Please upgrade numpy to >= 1.12.0 to use this pandas version

In [4]:
import pandas as pd

In [5]:
import numpy

In [6]:
from  pandora.astro import Mags

In [8]:
import matplotlib.plot

ImportError: No module named matplotlib.plot